In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import f1_score, r2_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")


%load_ext autoreload
%autoreload 2

In [2]:
data_sequence = pd.read_hdf('../data/preprocessed/data_sequence_alldata.hdf', key='final_alldata', mode='r')
data_sequence.shape

(167559, 1183)

In [3]:
with open('../data/preprocessed/hashs_train.pkl', 'rb') as fp:
    hashs_train = pickle.load(fp)
    
with open('../data/preprocessed/hashs_test.pkl', 'rb') as fp:
    hashs_test = pickle.load(fp)

In [4]:
train = data_sequence[data_sequence.hash.isin(hashs_train)]
test  = data_sequence[data_sequence.hash.isin(hashs_test)]

In [5]:
window_reference = 6

with open('../data/preprocessed/original_cols_sequence.pkl', 'rb') as fp: 
    original_cols = pickle.load(fp)
    
drop_cols = list(x for x in original_cols if 'exit' in x)# + grid_cols
drop_cols += ['lat_lon_entry', 'lat_lon_exit']
drop_cols += ['euclidean_distance', 'manhattan_distance', 'harvesine_distance',
              'center_permanency', 'crossed_city', 'velocity', 'leaving_city', 'entering_city']

drop_cols = [col+'_'+str(window_reference) for col in drop_cols]
#drop_cols += [col+'_'+str(i) for col in grid_cols for i in range(0, 5)]
drop_cols += [col for col in train.columns if col.endswith('exit_5')]
drop_cols += ['hash', f'delta_last_center_permanency_{window_reference}', f'delta_origin_center_permanency_{window_reference}']

features = list(set(train.columns) - set(drop_cols))
target   = ['is_inside_city_exit_'+str(window_reference)]

In [6]:
train_train, train_val = train_test_split(train[train.hour_exit_5>=15], test_size=0.25, random_state=423)
train_train.shape, train_val.shape

((100513, 1183), (33505, 1183))